In [ ]:
import gradio as gr
import subprocess
import os
import tempfile
from pathlib import Path

# 设置常量
PREDICT_SCRIPT = "predict.py"  # 你的预测脚本路径
MODEL_WEIGHTS = "best.pt"      # 你的模型权重路径

def process_video(input_video, conf_threshold=0.5, iou_threshold=0.5):
    """
    处理输入视频并返回跟踪结果
    
    参数:
        input_video: 上传的视频文件
        conf_threshold: 置信度阈值
        iou_threshold: IOU阈值
        
    返回:
        处理后的视频文件路径
    """
    # 创建临时目录保存结果
    output_dir = tempfile.mkdtemp()
    output_video = os.path.join(output_dir, "output.mp4")
    
    # 构建命令
    cmd = [
        "python", PREDICT_SCRIPT,
        "--weights", MODEL_WEIGHTS,
        "--source", input_video,
        "--output", output_video,
        "--conf-thres", str(conf_threshold),
        "--iou-thres", str(iou_threshold),
        "--save-vid"
    ]
    
    try:
        # 运行预测脚本
        subprocess.run(cmd, check=True)
        
        # 检查输出文件是否存在
        if os.path.exists(output_video):
            return output_video
        else:
            raise gr.Error("输出视频未生成，请检查预测脚本")
            
    except subprocess.CalledProcessError as e:
        raise gr.Error(f"处理视频时出错: {str(e)}")
    except Exception as e:
        raise gr.Error(f"发生错误: {str(e)}")

# 创建Gradio界面
with gr.Blocks(title="车辆跟踪系统") as demo:
    gr.Markdown("# 🚗 车辆跟踪系统")
    gr.Markdown("上传视频文件进行车辆检测与跟踪")
    
    with gr.Row():
        with gr.Column():
            video_input = gr.Video(label="输入视频", sources=["upload"])
            
            with gr.Accordion("高级设置", open=False):
                conf_slider = gr.Slider(
                    minimum=0.1, maximum=0.9, value=0.5, step=0.05,
                    label="置信度阈值"
                )
                iou_slider = gr.Slider(
                    minimum=0.1, maximum=0.9, value=0.5, step=0.05,
                    label="IOU阈值"
                )
            
            submit_btn = gr.Button("开始处理", variant="primary")
            
        with gr.Column():
            video_output = gr.Video(label="跟踪结果", autoplay=True)
    
    # 示例视频
    gr.Examples(
        examples=[["example1.mp4"], ["example2.mp4"]],
        inputs=video_input,
        label="示例视频 (点击试试)"
    )
    
    # 提交事件
    submit_btn.click(
        fn=process_video,
        inputs=[video_input, conf_slider, iou_slider],
        outputs=video_output
    )
    
    # 说明
    gr.Markdown("""
    ## 使用说明
    1. 上传视频文件或点击示例视频
    2. 调整参数(可选)
    3. 点击"开始处理"按钮
    4. 查看右侧的跟踪结果
    
    ## 技术细节
    - 使用YOLOv5模型进行车辆检测
    - 采用DeepSort算法进行目标跟踪
    - 输出视频包含边界框和跟踪ID
    """)

# 启动界面
if __name__ == "__main__":
    demo.launch(server_name="0.0.0.0", server_port=7860)